# read

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
import numpy as np; np.random.seed(0)
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/data/train_df.csv', encoding='cp949')
test  = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/data/test_df.csv', encoding='cp949')
sub   = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/data/sample_submission.csv', encoding='cp949')

In [2]:
train.shape, test.shape, sub.shape

((32000, 6), (4640, 5), (4640, 2))

In [3]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9
3,3,1154500002014200,5011000315087400,식품,농산물,10
4,4,1165000021008300,5011000177051200,식품,가공식품,3


# 격자공간고유번호 슬라이싱

In [4]:
for df in [train, test]:
    for i in range(1, 16):
        df['send_' + str(i)] = df['SEND_SPG_INNB'].astype('str').str[:i].astype('int64')
    for i in range(1, 16):
        df['rec_' + str(i)] = df['REC_SPG_INNB'].astype('str').str[:i].astype('int64')
    df['send_end_1'] = df['SEND_SPG_INNB'].astype('str').str[-6:-3]
    df['send_end_2'] = df['SEND_SPG_INNB'].astype('str').str[-3:]
    df['rec_end_1'] = df['REC_SPG_INNB'].astype('str').str[-6:-3]
    df['rec_end_2'] = df['REC_SPG_INNB'].astype('str').str[-3:]

In [5]:
for col in train.columns[6:]:
    print(f'{col} : {train[col].nunique()}')

send_1 : 5
send_2 : 17
send_3 : 64
send_4 : 213
send_5 : 234
send_6 : 234
send_7 : 240
send_8 : 511
send_9 : 1161
send_10 : 2069
send_11 : 2082
send_12 : 3081
send_13 : 3581
send_14 : 3684
send_15 : 3684
rec_1 : 5
rec_2 : 17
rec_3 : 65
rec_4 : 229
rec_5 : 251
rec_6 : 251
rec_7 : 268
rec_8 : 979
rec_9 : 3353
rec_10 : 7739
rec_11 : 7945
rec_12 : 19039
rec_13 : 26039
rec_14 : 27826
rec_15 : 27826
send_end_1 : 100
send_end_2 : 4
rec_end_1 : 100
rec_end_2 : 4


In [6]:
send_end_1_dict = {}
for i, j in zip(train['send_end_1'].value_counts().sort_values().index, range(100)):
    send_end_1_dict[i] = j

rec_end_1_dict = {}
for i, j in zip(train['rec_end_1'].value_counts().sort_values().index, range(100)):
    rec_end_1_dict[i] = j

send_end_2_dict = {}
for i, j in zip(train['send_end_2'].value_counts().sort_values().index, range(4)):
    send_end_2_dict[i] = j

rec_end_2_dict  = {}
for i, j in zip(train['rec_end_2'].value_counts().sort_values().index, range(4)):
    rec_end_2_dict[i] = j

print(send_end_1_dict)
print(send_end_2_dict)

{'002': 0, '019': 1, '020': 2, '093': 3, '092': 4, '083': 5, '081': 6, '042': 7, '064': 8, '027': 9, '087': 10, '056': 11, '035': 12, '046': 13, '048': 14, '038': 15, '033': 16, '084': 17, '040': 18, '061': 19, '071': 20, '039': 21, '026': 22, '016': 23, '050': 24, '001': 25, '032': 26, '066': 27, '025': 28, '076': 29, '088': 30, '086': 31, '014': 32, '051': 33, '013': 34, '053': 35, '059': 36, '089': 37, '100': 38, '098': 39, '044': 40, '023': 41, '012': 42, '095': 43, '074': 44, '082': 45, '080': 46, '075': 47, '009': 48, '097': 49, '077': 50, '085': 51, '062': 52, '058': 53, '069': 54, '008': 55, '007': 56, '034': 57, '021': 58, '005': 59, '043': 60, '003': 61, '047': 62, '065': 63, '045': 64, '054': 65, '037': 66, '041': 67, '073': 68, '029': 69, '052': 70, '011': 71, '070': 72, '004': 73, '079': 74, '006': 75, '018': 76, '031': 77, '036': 78, '091': 79, '022': 80, '072': 81, '060': 82, '096': 83, '010': 84, '024': 85, '094': 86, '057': 87, '078': 88, '090': 89, '067': 90, '099': 9

In [7]:
for df in [train, test]:
    df['send_end_1_cnt'] = df['send_end_1'].map(send_end_1_dict)
    df['rec_end_1_cnt']  = df['rec_end_1'].map(rec_end_1_dict)
    df['send_end_2_cnt'] = df['send_end_2'].map(send_end_2_dict)
    df['rec_end_2_cnt']  = df['rec_end_2'].map(send_end_2_dict)

In [8]:
send_label = dict(train.groupby('send_end_1').mean().sort_values('INVC_CONT')[['INVC_CONT']].reset_index().values)
rec_label  = dict(train.groupby('rec_end_1').mean().sort_values('INVC_CONT')[['INVC_CONT']].reset_index().values)

for df in [train, test]:
    df['send_end_1'] = df['send_end_1'].map(send_label)
    df['rec_end_1']  = df['rec_end_1'].map(rec_label)

In [9]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,send_1,send_2,send_3,send_4,send_5,send_6,send_7,send_8,send_9,send_10,send_11,send_12,send_13,send_14,send_15,rec_1,rec_2,rec_3,rec_4,rec_5,rec_6,rec_7,rec_8,rec_9,rec_10,rec_11,rec_12,rec_13,rec_14,rec_15,send_end_1,send_end_2,rec_end_1,rec_end_2,send_end_1_cnt,rec_end_1_cnt,send_end_2_cnt,rec_end_2_cnt
0,0,1129000014045300,5011000220046300,패션의류,상의,3,1,11,112,1129,11290,112900,1129000,11290000,112900001,1129000014,11290000140,112900001404,1129000014045,11290000140453,112900001404530,5,50,501,5011,50110,501100,5011000,50110002,501100022,5011000220,50110002200,501100022004,5011000220046,50110002200463,501100022004630,5.102679,300,4.296667,300,64,19,0,0
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3,1,11,113,1135,11350,113500,1135000,11350000,113500000,1135000009,11350000090,113500000905,1135000009051,11350000090512,113500000905120,5,50,501,5011,50110,501100,5011000,50110001,501100017,5011000178,50110001780,501100017803,5011000178037,50110001780373,501100017803730,5.137681,200,4.375940,300,33,1,1,0
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9,1,11,113,1135,11350,113500,1135000,11350000,113500003,1135000030,11350000300,113500003009,1135000030093,11350000300931,113500003009310,5,50,501,5011,50110,501100,5011000,50110002,501100026,5011000265,50110002650,501100026509,5011000265091,50110002650914,501100026509140,5.039474,100,4.723926,400,3,56,3,2
3,3,1154500002014200,5011000315087400,식품,농산물,10,1,11,115,1154,11545,115450,1154500,11545000,115450000,1154500002,11545000020,115450000201,1154500002014,11545000020142,115450000201420,5,50,501,5011,50110,501100,5011000,50110003,501100031,5011000315,50110003150,501100031508,5011000315087,50110003150874,501100031508740,4.876812,200,4.709571,400,32,25,1,2
4,4,1165000021008300,5011000177051200,식품,가공식품,3,1,11,116,1165,11650,116500,1165000,11650000,116500002,1165000021,11650000210,116500002100,1165000021008,11650000210083,116500002100830,5,50,501,5011,50110,501100,5011000,50110001,501100017,5011000177,50110001770,501100017705,5011000177051,50110001770512,501100017705120,5.591160,300,4.335505,200,55,29,0,1


In [10]:
drop_cols = [
    'send_1', 'send_2', 'send_4', 'send_5', 'send_6', 
    'send_10', 'send_14', 'send_15', 'rec_1', 'rec_2', 
    'rec_4', 'rec_5', 'rec_6', 'rec_10', 'rec_13', 
    'rec_14', 'rec_15', 'send_end_2', 'rec_end_2'
]

# 유사지역 운송

In [11]:
for i in range(1, 16):
    train_sim = train[train[f'send_{i}'] == train[f'rec_{i}']].shape[0]
    test_sim  = test[test[f'send_{i}'] == test[f'rec_{i}']].shape[0]
    print(f'send_rec_{i} : train {train_sim}, test {test_sim}')

send_rec_1 : train 177, test 28
send_rec_2 : train 177, test 28
send_rec_3 : train 177, test 28
send_rec_4 : train 132, test 19
send_rec_5 : train 132, test 19
send_rec_6 : train 132, test 19
send_rec_7 : train 129, test 17
send_rec_8 : train 68, test 12
send_rec_9 : train 56, test 6
send_rec_10 : train 49, test 5
send_rec_11 : train 49, test 5
send_rec_12 : train 39, test 2
send_rec_13 : train 39, test 2
send_rec_14 : train 37, test 2
send_rec_15 : train 37, test 2


In [12]:
for df in [train, test]:
    for i in [3, 7, 8, 10]:
        df[f'sim_INNB_{i}'] = 0
        df.loc[df[f'send_{i}'] == df[f'rec_{i}'], f'sim_INNB_{i}'] = 1

In [13]:
train.loc[:, 'sim_INNB_3':].corr()

,sim_INNB_3,sim_INNB_7,sim_INNB_8,sim_INNB_10
sim_INNB_3,1.000000,0.853063,0.618764,0.525097
sim_INNB_7,0.853063,1.000000,0.725345,0.615544
sim_INNB_8,0.618764,0.725345,1.000000,0.848622
sim_INNB_10,0.525097,0.615544,0.848622,1.000000


# 지역별 운송 빈도

In [14]:
for col in train.columns[6:-4]:
    print(f'##### {col} value_counts')
    print(train[col].value_counts())
    print('-----------------------------')

##### send_1 value_counts
5    25344
4     4984
1      859
2      550
3      263
Name: send_1, dtype: int64
-----------------------------
##### send_2 value_counts
50    25344
41     3950
11      859
30      231
26      187
27      185
43      184
44      175
47      171
45      163
28      132
46      124
42      118
48       99
29       46
31       28
36        4
Name: send_2, dtype: int64
-----------------------------
##### send_3 value_counts
501    25344
414     1887
415      909
412      358
413      352
       ...  
458        4
479        4
422        3
469        2
312        1
Name: send_3, Length: 64, dtype: int64
-----------------------------
##### send_4 value_counts
5011    14003
5013    11341
4148     1203
4159      395
4141      363
        ...  
4672        1
4221        1
4182        1
4129        1
4792        1
Name: send_4, Length: 213, dtype: int64
-----------------------------
##### send_5 value_counts
50110    14003
50130    11341
41480     1203
41590      395
4

In [15]:
train['send_2'].value_counts()

50    25344
41     3950
11      859
30      231
26      187
27      185
43      184
44      175
47      171
45      163
28      132
46      124
42      118
48       99
29       46
31       28
36        4
Name: send_2, dtype: int64

In [16]:
send_freq2_map = {11: 1, 41: 2, 50: 3}

for df in [train, test]:
    df['send_freq_2'] = df['send_2'].map(send_freq2_map)
    df['send_freq_2'].fillna(0, inplace=True)

In [17]:
train['rec_2'].value_counts()

41    7061
50    6833
11    4984
26    1548
28    1419
48    1397
44    1193
47    1138
45     942
27     911
46     885
42     874
43     770
30     682
29     636
31     544
36     183
Name: rec_2, dtype: int64

In [18]:
rec_freq2_map = {47: 1, 44: 1, 48: 1, 28: 1, 26: 1, 11: 2, 50: 3, 41: 3}

for df in [train, test]:
    df['rec_freq_2'] = df['rec_2'].map(rec_freq2_map)
    df['rec_freq_2'].fillna(0, inplace=True)

In [19]:
for df in [train, test]:
    df[['send_freq_2', 'rec_freq_2']] = df[['send_freq_2', 'rec_freq_2']].astype('int64')

# 분류별 target 평균

In [20]:
large_label = dict(train.groupby('DL_GD_LCLS_NM').mean().sort_values('INVC_CONT')[['INVC_CONT']].reset_index().values)
mid_label   = dict(train.groupby('DL_GD_MCLS_NM').mean().sort_values('INVC_CONT')[['INVC_CONT']].reset_index().values)

for df in [train, test]:
    df['DL_GD_LCLS_NM'] = df['DL_GD_LCLS_NM'].map(large_label)
    df['DL_GD_MCLS_NM'] = df['DL_GD_MCLS_NM'].map(mid_label)

In [21]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,send_1,send_2,send_3,send_4,send_5,send_6,send_7,send_8,send_9,send_10,send_11,send_12,send_13,send_14,send_15,rec_1,rec_2,rec_3,rec_4,rec_5,rec_6,rec_7,rec_8,rec_9,rec_10,rec_11,rec_12,rec_13,rec_14,rec_15,send_end_1,send_end_2,rec_end_1,rec_end_2,send_end_1_cnt,rec_end_1_cnt,send_end_2_cnt,rec_end_2_cnt,sim_INNB_3,sim_INNB_7,sim_INNB_8,sim_INNB_10,send_freq_2,rec_freq_2
0,0,1129000014045300,5011000220046300,6.678694,3.672897,3,1,11,112,1129,11290,112900,1129000,11290000,112900001,1129000014,11290000140,112900001404,1129000014045,11290000140453,112900001404530,5,50,501,5011,50110,501100,5011000,50110002,501100022,5011000220,50110002200,501100022004,5011000220046,50110002200463,501100022004630,5.102679,300,4.296667,300,64,19,0,0,0,0,0,0,1,3
1,1,1135000009051200,5011000178037300,5.407921,4.444134,3,1,11,113,1135,11350,113500,1135000,11350000,113500000,1135000009,11350000090,113500000905,1135000009051,11350000090512,113500000905120,5,50,501,5011,50110,501100,5011000,50110001,501100017,5011000178,50110001780,501100017803,5011000178037,50110001780373,501100017803730,5.137681,200,4.375940,300,33,1,1,0,0,0,0,0,1,3
2,2,1135000030093100,5011000265091400,6.678694,10.375479,9,1,11,113,1135,11350,113500,1135000,11350000,113500003,1135000030,11350000300,113500003009,1135000030093,11350000300931,113500003009310,5,50,501,5011,50110,501100,5011000,50110002,501100026,5011000265,50110002650,501100026509,5011000265091,50110002650914,501100026509140,5.039474,100,4.723926,400,3,56,3,2,0,0,0,0,1,3
3,3,1154500002014200,5011000315087400,4.658195,4.297401,10,1,11,115,1154,11545,115450,1154500,11545000,115450000,1154500002,11545000020,115450000201,1154500002014,11545000020142,115450000201420,5,50,501,5011,50110,501100,5011000,50110003,501100031,5011000315,50110003150,501100031508,5011000315087,50110003150874,501100031508740,4.876812,200,4.709571,400,32,25,1,2,0,0,0,0,1,3
4,4,1165000021008300,5011000177051200,4.658195,6.807151,3,1,11,116,1165,11650,116500,1165000,11650000,116500002,1165000021,11650000210,116500002100,1165000021008,11650000210083,116500002100830,5,50,501,5011,50110,501100,5011000,50110001,501100017,5011000177,50110001770,501100017705,5011000177051,50110001770512,501100017705120,5.591160,300,4.335505,200,55,29,0,1,0,0,0,0,1,3


# drop

In [22]:
drop_cols = drop_cols + ['index', 'SEND_SPG_INNB', 'REC_SPG_INNB']

for df in [train, test]:
    df.drop(drop_cols, axis=1, inplace=True)

In [23]:
train.head()

,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,send_3,send_7,send_8,send_9,send_11,send_12,send_13,rec_3,rec_7,rec_8,rec_9,rec_11,rec_12,send_end_1,rec_end_1,send_end_1_cnt,rec_end_1_cnt,send_end_2_cnt,rec_end_2_cnt,sim_INNB_3,sim_INNB_7,sim_INNB_8,sim_INNB_10,send_freq_2,rec_freq_2
0,6.678694,3.672897,3,112,1129000,11290000,112900001,11290000140,112900001404,1129000014045,501,5011000,50110002,501100022,50110002200,501100022004,5.102679,4.296667,64,19,0,0,0,0,0,0,1,3
1,5.407921,4.444134,3,113,1135000,11350000,113500000,11350000090,113500000905,1135000009051,501,5011000,50110001,501100017,50110001780,501100017803,5.137681,4.375940,33,1,1,0,0,0,0,0,1,3
2,6.678694,10.375479,9,113,1135000,11350000,113500003,11350000300,113500003009,1135000030093,501,5011000,50110002,501100026,50110002650,501100026509,5.039474,4.723926,3,56,3,2,0,0,0,0,1,3
3,4.658195,4.297401,10,115,1154500,11545000,115450000,11545000020,115450000201,1154500002014,501,5011000,50110003,501100031,50110003150,501100031508,4.876812,4.709571,32,25,1,2,0,0,0,0,1,3
4,4.658195,6.807151,3,116,1165000,11650000,116500002,11650000210,116500002100,1165000021008,501,5011000,50110001,501100017,50110001770,501100017705,5.591160,4.335505,55,29,0,1,0,0,0,0,1,3


In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32000 entries, 0 to 31999
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DL_GD_LCLS_NM   32000 non-null  float64
 1   DL_GD_MCLS_NM   32000 non-null  float64
 2   INVC_CONT       32000 non-null  int64  
 3   send_3          32000 non-null  int64  
 4   send_7          32000 non-null  int64  
 5   send_8          32000 non-null  int64  
 6   send_9          32000 non-null  int64  
 7   send_11         32000 non-null  int64  
 8   send_12         32000 non-null  int64  
 9   send_13         32000 non-null  int64  
 10  rec_3           32000 non-null  int64  
 11  rec_7           32000 non-null  int64  
 12  rec_8           32000 non-null  int64  
 13  rec_9           32000 non-null  int64  
 14  rec_11          32000 non-null  int64  
 15  rec_12          32000 non-null  int64  
 16  send_end_1      32000 non-null  float64
 17  rec_end_1       32000 non-null 

# corr

In [25]:
train.corr()

,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,send_3,send_7,send_8,send_9,send_11,send_12,send_13,rec_3,rec_7,rec_8,rec_9,rec_11,rec_12,send_end_1,rec_end_1,send_end_1_cnt,rec_end_1_cnt,send_end_2_cnt,rec_end_2_cnt,sim_INNB_3,sim_INNB_7,sim_INNB_8,sim_INNB_10,send_freq_2,rec_freq_2
DL_GD_LCLS_NM,1.000000,0.342224,0.062190,-0.451482,-0.451231,-0.451231,-0.451231,-0.451231,-0.451231,-0.451231,0.203205,0.202902,0.202902,0.202902,0.202902,0.202902,0.154341,0.011786,-0.096618,0.004881,-0.066535,-0.007355,-0.004301,-0.000561,-0.004987,-0.011335,-0.370076,0.242896
DL_GD_MCLS_NM,0.342224,1.000000,0.181723,-0.375542,-0.375489,-0.375489,-0.375489,-0.375489,-0.375489,-0.375489,0.234644,0.234332,0.234332,0.234332,0.234332,0.234332,0.307683,0.025052,-0.262387,0.005597,-0.085513,-0.005837,0.059245,0.062619,0.041706,0.039097,-0.420906,0.228823
INVC_CONT,0.062190,0.181723,1.000000,-0.058765,-0.058702,-0.058702,-0.058702,-0.058702,-0.058702,-0.058702,0.063267,0.063201,0.063201,0.063201,0.063201,0.063201,0.173425,0.056744,-0.102596,0.009383,-0.025896,0.009708,0.015316,0.013114,0.017200,0.018531,-0.088158,0.036048
send_3,-0.451482,-0.375542,-0.058765,1.000000,0.999997,0.999997,0.999997,0.999997,0.999997,0.999997,-0.376118,-0.375639,-0.375639,-0.375639,-0.375639,-0.375639,-0.271316,-0.018082,0.218842,-0.015197,0.096147,0.003900,0.028495,0.024308,0.017632,0.014963,0.733644,-0.377887
send_7,-0.451231,-0.375489,-0.058702,0.999997,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.375697,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.271158,-0.018065,0.218880,-0.015181,0.095629,0.003879,0.028429,0.024229,0.017583,0.014924,0.733314,-0.377442
send_8,-0.451231,-0.375489,-0.058702,0.999997,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.375697,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.271158,-0.018065,0.218880,-0.015181,0.095629,0.003879,0.028429,0.024229,0.017583,0.014924,0.733314,-0.377442
send_9,-0.451231,-0.375489,-0.058702,0.999997,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.375697,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.271158,-0.018065,0.218880,-0.015181,0.095629,0.003879,0.028429,0.024229,0.017583,0.014924,0.733314,-0.377442
send_11,-0.451231,-0.375489,-0.058702,0.999997,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.375697,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.271158,-0.018065,0.218880,-0.015181,0.095629,0.003879,0.028429,0.024229,0.017583,0.014924,0.733314,-0.377442
send_12,-0.451231,-0.375489,-0.058702,0.999997,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.375697,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.271158,-0.018065,0.218880,-0.015181,0.095629,0.003879,0.028429,0.024229,0.017583,0.014924,0.733314,-0.377442
send_13,-0.451231,-0.375489,-0.058702,0.999997,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.375697,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.271158,-0.018065,0.218880,-0.015181,0.095629,0.003879,0.028429,0.024229,0.017583,0.014924,0.733314,-0.377442


# modeling

In [26]:
X = train.drop('INVC_CONT', axis=1)
y = train['INVC_CONT']

In [27]:
!pip install catboost

In [28]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(random_state=0, random_strength=0)

model.fit(X, y)
pd.DataFrame({'col':X.columns, 'fi':model.feature_importances_})

Learning rate set to 0.070793
0:	learn: 5.7039652	total: 78.6ms	remaining: 1m 18s
1:	learn: 5.6588458	total: 111ms	remaining: 55.4s
2:	learn: 5.6184239	total: 139ms	remaining: 46.2s
3:	learn: 5.5795004	total: 162ms	remaining: 40.4s
4:	learn: 5.5426795	total: 192ms	remaining: 38.2s
5:	learn: 5.5035836	total: 233ms	remaining: 38.6s
6:	learn: 5.4713186	total: 258ms	remaining: 36.6s
7:	learn: 5.4424138	total: 283ms	remaining: 35.1s
8:	learn: 5.4139856	total: 314ms	remaining: 34.5s
9:	learn: 5.3908849	total: 348ms	remaining: 34.4s
10:	learn: 5.3666649	total: 376ms	remaining: 33.8s
11:	learn: 5.3421956	total: 412ms	remaining: 33.9s
12:	learn: 5.3209101	total: 425ms	remaining: 32.3s
13:	learn: 5.3043043	total: 444ms	remaining: 31.3s
14:	learn: 5.2853928	total: 479ms	remaining: 31.5s
15:	learn: 5.2665458	total: 509ms	remaining: 31.3s
16:	learn: 5.2497316	total: 519ms	remaining: 30s
17:	learn: 5.2328962	total: 532ms	remaining: 29s
18:	learn: 5.2175819	total: 543ms	remaining: 28.1s
19:	learn: 5.

,col,fi
0,DL_GD_LCLS_NM,1.420368
1,DL_GD_MCLS_NM,21.128188
2,send_3,0.597904
3,send_7,2.845860
4,send_8,1.617291
5,send_9,1.733112
6,send_11,1.792615
7,send_12,1.052559
8,send_13,1.019825
9,rec_3,0.387384


In [29]:
from sklearn.model_selection import GridSearchCV

model = CatBoostRegressor(random_state=0, random_strength=0)

gscv = GridSearchCV(
    estimator=model, param_grid={}, 
    scoring='neg_root_mean_squared_error', 
    refit=True, cv=10
)

gscv.fit(X, y)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
4:	learn: 5.6061742	total: 42.8ms	remaining: 8.51s
5:	learn: 5.5772511	total: 51.3ms	remaining: 8.51s
6:	learn: 5.5404623	total: 74.1ms	remaining: 10.5s
7:	learn: 5.5074066	total: 83.3ms	remaining: 10.3s
8:	learn: 5.4756446	total: 92.4ms	remaining: 10.2s
9:	learn: 5.4464473	total: 102ms	remaining: 10.1s
10:	learn: 5.4240284	total: 111ms	remaining: 9.95s
11:	learn: 5.3981868	total: 120ms	remaining: 9.84s
12:	learn: 5.3745070	total: 128ms	remaining: 9.74s
13:	learn: 5.3516742	total: 138ms	remaining: 9.7s
14:	learn: 5.3289769	total: 147ms	remaining: 9.63s
15:	learn: 5.3097997	total: 156ms	remaining: 9.57s
16:	learn: 5.2914105	total: 165ms	remaining: 9.52s
17:	learn: 5.2729782	total: 173ms	remaining: 9.45s
18:	learn: 5.2577095	total: 182ms	remaining: 9.38s
19:	learn: 5.2434034	total: 191ms	remaining: 9.34s
20:	learn: 5.2285641	total: 207ms	remaining: 9.66s
21:	learn: 5.2142691	total: 219ms	remaining: 9.71s
22:	learn: 5.1998325	total: 229ms	remaining: 9.7

GridSearchCV(cv=10,
             estimator=<catboost.core.CatBoostRegressor object at 0x7fb7eef64d50>,
             param_grid={}, scoring='neg_root_mean_squared_error')

# sub

In [30]:
sub['INVC_CONT'] = gscv.predict(test)
best = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/sub/logistics_11.csv')
sr   = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/sub/soryoung_submission_v36.csv')

In [31]:
pd.concat([sub, best['INVC_CONT'], sr['INVC_CONT']], axis=1)

,index,INVC_CONT,INVC_CONT,INVC_CONT
0,32000,4.170143,4.751713,4.674949
1,32001,4.408378,4.881143,4.735952
2,32002,5.428293,5.283379,5.289564
3,32003,6.146122,6.022410,6.127594
4,32004,5.942313,5.570852,5.293542
...,...,...,...,...
4635,36635,4.717508,7.302492,8.644497
4636,36636,5.135318,5.042661,5.057579
4637,36637,3.981829,4.788176,4.777744
4638,36638,4.855498,4.705686,4.648363


In [32]:
from sklearn.metrics import mean_squared_error

best_sr  = mean_squared_error(sr['INVC_CONT'], best['INVC_CONT'])
sub_sr   = mean_squared_error(sr['INVC_CONT'], sub['INVC_CONT'])
sub_best = mean_squared_error(best['INVC_CONT'], sub['INVC_CONT'])

best_sr, sub_sr, sub_best

(1.1975378200115878, 7.182237509687671, 6.093313212500342)

In [33]:
sub.to_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/sub/logistics_13.csv', index=False)